In [3]:
from models import *
import pandas as pd
import numpy as np
import os

## Load Training Data

In [4]:
train = pd.read_csv('tf_lemmatized_initial_feature_set.csv')
train.head()

,Unnamed: 0,index,text,label,score,comp_score,pos_score,000,10,100,...,yes,yet,york,you,young,younger,your,youth,zero,zombie
0,0,0,"['welcome', 'survive', 'tried', 'life', '!', '...",pos,"['Bromwell High is a cartoon comedy.', 'It ran...",-0.012378,0.090444,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,1,1,"['sailor', 'goddard', 'live', 'carlin', 'house...",pos,['Homelessness (or Houselessness as George Car...,0.110767,0.084000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.044742,0.0,0.0,0.0,0.0,0.0
2,2,2,"['second', 'three', 'look', 'tambor', 'blazing...",pos,['Brilliant over-acting by Lesley Ann Warren.'...,0.248927,0.219727,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,3,3,"['fisher', 'flawed', 'sure', 'society', 'lovab...",pos,['This is easily the most underrated film inn ...,0.392357,0.252714,0.0,0.0,0.0,...,0.0,0.0,0.0,0.082231,0.000000,0.0,0.0,0.0,0.0,0.0
4,4,4,"['sailor', 'actress', 'film', 'job', 'actually...",pos,"['This is not the typical Mel Brooks film.', '...",0.378333,0.152167,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [5]:
# Shuffle data prior to building model
train = train.sample(frac = 1)

# Make a numeric label for model building (LDA and SVM)
train['lda_numeric_label'] = train.apply(lambda x: 1 if x['label'] == 'pos' else -1, axis = 1)
# # For Logistic
train['log_numeric_label'] = train.apply(lambda x: 1 if x['label'] == 'pos' else 0, axis = 1)


model_response = train[['log_numeric_label']]
# Currently only using categorical variables
model_predictors = pd.DataFrame(train.iloc[:,5:])

In [10]:
def cross_validation(x, y, fold_num = 5, metrics = ['accuracy']):
    """
    x - predictor variables in Pandas DataFrame
    y - response variables in Pandas DataFrame
    fold_num - number of folds for k fold cross validation
    
    perform k fold cross validation and report the average for our metric of interest over the k folds
    """
        
    num_rows = x.shape[0]
    rows_per_fold = num_rows // fold_num
    
    # Is used later to calculate training and testing indices
    total_indices = set(range(num_rows))
    
    
    log_model_eval = {metric: [] for metric in metrics}

    # Iterate over the data set
    for fold_count in range(0,fold_num):
        # Calculate the split
        test_indices = list(set(range(fold_count*rows_per_fold,(fold_count*rows_per_fold)  + rows_per_fold)))
        train_indices = list(set(total_indices) - set(test_indices))

        # Perform the split
        train_x = x.iloc[train_indices,:]
        train_y = y.iloc[train_indices,]
                
        test_x = x.iloc[test_indices,:]
        test_y = y.iloc[test_indices]
        
#         # Fit models to training data
#         lda_model = LDA()
#         lda_model.fit(train_x.to_numpy(), train_y.to_numpy())
        
        log_model = LogReg()
        log_model.fit(train_x, train_y)
        
        
        for metric in  metrics:
            if metric == 'accuracy':
                model_eval = pd.DataFrame({'pred':log_model.predict(test_x),'ground_truth':test_y.log_numeric_label})
                log_model_eval[metric].append(sum(model_eval.pred == model_eval.ground_truth) / model_eval.shape[0])
                print(log_model_eval)
#                 model_eval = pd.DataFrame({'pred':lda_model.predict(test_x.to_numpy()),'ground_truth':test_y.numeric_label})
#                 log_model_eval[metric].append(sum(model_eval.pred == model_eval.ground_truth) / model_eval.shape[0])

            else:
                pass

    print(log_model_eval)
    return [metric + ' ' +str(sum(log_model_eval[metric])/fold_num) for metric in metrics]

In [ ]:
cross_validation(model_predictors, model_response,2)

1
2


In [12]:
from sklearn.linear_model import LogisticRegression

# Perform the split
train_x = model_predictors.iloc[:1000,:]
train_y = model_response.iloc[:1000,]

test_x = model_predictors.iloc[1001:1300,:]
test_y = model_response.iloc[1001:1300,]
log = LogisticRegression()
log.fit(train_x, train_y)

c:\users\gjber\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
log_model = LogReg()
log_model.fit(train_x, train_y)

1
2
3
4
5
6


In [14]:
model_eval = pd.DataFrame({'pred':log_model.predict(test_x),'ground_truth':test_y.log_numeric_label})
sum(model_eval.pred == model_eval.ground_truth) / model_eval.shape[0]

1.0